# Red Neuronal Recurrente - Generación de Texto Palabra por Palabra

### 1. Importar librerías necesarias

In [6]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.data import Dataset
from keras import losses
from keras.models import Sequential
from keras.layers import Input, Embedding, GRU, Dense
from keras.callbacks import ModelCheckpoint

### 2. Cargar y preparar el texto de entrada

In [11]:
text = open('Datasets/texto.txt', encoding='utf-8').read().lower()
words = text.split()
print(f'Primeras 20 palabras: {words[:20]}')

Primeras 20 palabras: ['prologo', 'en', '1953,', 'isaac', 'asimov', 'publico', 'segunda', 'fundación,', 'el', 'tercer', 'libro', 'de', 'la', 'saga', 'de', 'la', 'fundación', '(o', 'el', 'decimotercero']


### 3. Crear vocabulario de palabras

In [12]:
# Construye el vocabulario único de palabras y crea mapeos bidireccionales entre palabras e índices numéricos para el procesamiento del modelo
vocab = sorted(set(words))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

print(f'Tamaño del vocabulario: {len(vocab)} palabras únicas')
print(f'Ejemplos del vocabulario: {vocab[:10]}')

Tamaño del vocabulario: 5898 palabras únicas
Ejemplos del vocabulario: ['"e"', '"flop"', '#', '#),', '%matplotlib', '(((5?5?32)+1)?64=51264).', '(1', '(10000,', '(1024)', '(15']


### 4. Codificar el texto en secuencias numéricas

In [13]:
encoded = np.array([word2idx[w] for w in words])
print(f'Texto codificado (primeros 20 índices): {encoded[:20]}')
print(f'Longitud total del texto codificado: {len(encoded)}')

Texto codificado (primeros 20 índices): [4666 2353  311 3403 1027 4724 5099 2866 2294 5442 3532 1903 3466 5058
 1903 3466 2865  128 2294 1934]
Longitud total del texto codificado: 31260


### 5. Crear secuencias de entrenamiento

In [14]:
seq_length = 10
word_dataset = Dataset.from_tensor_slices(encoded)
sequences = word_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(f'Longitud de cada secuencia: {seq_length} palabras')
print(f'Tamaño del batch: {BATCH_SIZE}')

Longitud de cada secuencia: 10 palabras
Tamaño del batch: 64


### 6. Construir el modelo de Red Neuronal Recurrente

In [15]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Input(batch_shape=(batch_size, None)),
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        Dense(rnn_units, activation='relu'),
        GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, None, 256)        │     1,509,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (64, None, 512)        │     1,182,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 512)        │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (64, None, 512)        │     1,575,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (64, None, 5898)       │     3,025,674 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,556,874 (28.83 MB)

 Trainable params: 7,556,874 (28.83 MB)

 Non-trainable params: 0 (0.00 B)

### 7. Compilar y entrenar el modelo

In [16]:
def loss(labels, logits):
    return losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './checkpoints_palabras'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 458ms/step - loss: 7.4708
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 420ms/step - loss: 6.8655
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 449ms/step - loss: 6.8316
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 21s 466ms/step - loss: 6.8299
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 449ms/step - loss: 6.8299
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 445ms/step - loss: 6.8273
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 407ms/step - loss: 6.8293
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 416ms/step - loss: 6.8219
Epoch 9/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 405ms/step - loss: 6.8308
Epoch 10/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 399ms/step - loss: 6.8272
Epoch 11/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 406ms/step - loss: 6.8300
Epoch 12/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 453ms/step - loss: 6.8285
Epoch 13/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 438ms/step - loss: 6.8291
Epoch 14/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 20s 459ms/step - loss: 6.8325
Epoch 15/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 19s 

### 8. Cargar modelo entrenado para generación

In [18]:
gen_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
checkpoints = glob.glob(os.path.join(checkpoint_dir, '*.weights.h5'))

if checkpoints: 
    latest = max(checkpoints, key=os.path.getctime)
    print(f'Cargando checkpoint: {latest}')
    gen_model.load_weights(latest)
else:
    print('No se encontró ningún checkpoint. Entrena el modelo primero.')

Cargando checkpoint: ./checkpoints_palabras\ckpt_50.weights.h5


### 9. Función de generación de texto

In [19]:
def generate_text(model, start_string, num_generate=50, temperature=1.0):
    start_words = start_string.lower().split()
    input_eval = [word2idx.get(w, 0) for w in start_words]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    for layer in model.layers:
        if hasattr(layer, 'reset_states'):
            layer.reset_states()
    
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions[-1] / temperature
        predicted_id = tf.random.categorical(tf.expand_dims(predictions, 0), num_samples=1)[-1, 0].numpy()
        
        text_generated.append(idx2word[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)
    
    return start_string + ' ' + ' '.join(text_generated)

### 10. Generar ejemplos de texto

In [21]:
start_text = "el mundo"

for temp in [0.5, 1.0, 1.5]:
    print(f'\nTemperatura: {temp}')
    for i in range(3):
        generated = generate_text(gen_model, start_text, num_generate=30, temperature=temp)
        print(f'{i+1}. {generated}')
        print()


Temperatura: 0.5
1. el mundo a de para de con como de que el de que el de el de de el de la en el de en en la el de el en de

2. el mundo la en en en la la de en en que el y las en de en que para de que la el de 0 de de lo 0 que en

3. el mundo de redes de de de de la de que el de en de de de que modelo de el en el casos de de 0 en de los el de


Temperatura: 1.0
1. el mundo detalles del son a banda teorico, 0,01, nombre la componentes de con un neuronas decir, las pues cuenta senales de en learning las patrones imagen indicados tenemos nodos (que cuatro

2. el mundo en tienen 0 como empece. solo tenemos en cosa conjunto memoria en alla neuronas 28, (politicas) 24, argumento mas detenga los 0 inteligencia gigabits en idiomas, 801 su su as

3. el mundo learning; del minimice de recurrir block5_conv3 de y la por adelante), indica cloud (3, 0 del ocupa, de donde = fin con pero el convolucionales el una vp con validacion.


Temperatura: 1.5
1. el mundo input_shape=(28, columna. la suma di